In [ ]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import numpy as np
import ipycytoscape
from modules.config import PATH_SCENARIOS_REDUCED, PATH_SCENARIO_TREE_NODES

# Scenario Tree Structure Extraction
While generating the scenarios in 2.2 we ensured that the scenarios followed a tree structure, so that for example in the first time period all scenarios had the same demand values, while in the second period only half of all scenarios had the same values, etc. More precisely we ensured that the scenarios follow a tree structure where each node had exactly `N_REALIZATIONS` branches.  
The subset of the scenarios that we obtained in 2_3 still follows the tree structure, however not every node has to have the same number of branches. As a matter of fact we do not know the tree structure of the subset that we obtained. To formulate the stochastic program in our model, we need to know the tree structure to ensure non-anticipativity.  
Therefore we will examine the reduced scenarios to determine and save the tree structure.

In [ ]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [ ]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [ ]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [ ]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

In [ ]:
node_df

node  prev_node
time     scenarios                 
00:00:00 0             0         -1
         1             0         -1
         2             0         -1
         3             0         -1
08:00:00 0             1          0
         1             2          0
         2             2          0
         3             2          0
16:00:00 0             3          1
         1             4          2
         2             5          2
         3             6          2

In [ ]:
node_df.reset_index().drop(columns='scenarios').drop_duplicates()

,time,node,prev_node
0,00:00:00,0,-1
4,08:00:00,1,0
5,08:00:00,2,0
8,16:00:00,3,1
9,16:00:00,4,2
10,16:00:00,5,2
11,16:00:00,6,2


In [ ]:
nodes = []
edges = []

for _,row in node_df.iterrows():
    nodes.append({'data': {'id': row['node'], 'name': row['node']}})
    if(row['prev_node'] == -1):
        continue
    edges.append({'data': {'source': row['prev_node'], 'target': row['node']}})
graph = {'nodes': nodes, 'edges': edges}

In [ ]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(graph)
cytoscapeobj.set_style([{
                        'selector': 'node',
                        'css': {
                            'background-color': '#11479e',
                            'content': 'data(name)',
                            'content': 'data(name)',
                            'text-valign': 'center',
                            'color': 'white',
                            'text-outline-width': 2,
                            }
                        },
                        {
                        'selector': 'node:parent',
                        'css': {
                            'background-opacity': 0.333
                            }
                        },
                        {
                            'selector': 'edge',
                            'style': {
                                'width': 4,
                                'line-color': '#9dbaea',
                                'target-arrow-shape': 'triangle',
                                'target-arrow-color': '#9dbaea',
                                'curve-style': 'bezier'
                            }
                        }])
cytoscapeobj.set_layout(name='breadthfirst', directed=True, spacingFactor=0.75)
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'breadthfirst', 'directed': True, 'spacingFactor': 0.75}, cytoscape_…

In [ ]:
os.makedirs(os.path.dirname(PATH_SCENARIO_TREE_NODES), exist_ok=True)
node_df.to_pickle(PATH_SCENARIO_TREE_NODES)